# 数据处理：清洗、准备和规整

在数据分析和建模的过程中，相当多的时间要用在数据准备上：加载、清理、转换以及重塑。这些工作会占到分析师时间的80%或更多。有时，存储在文件和数据库中的数据的格式不适合某个特定的任务。

在本章中，讨论处理缺失数据、重复数据、字符串操作和其它分析数据转换的工具，以及使用多种方法合并、重塑数据集。

## 第7章
## 7.1 处理缺失数据

对于数值数据，pandas使用浮点值NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以使用isnull()方法检测出来：

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull() #检测缺失值

0    False
1    False
2     True
3    False
dtype: bool

### 7.1.1 滤除缺失数据

对于DF对象，dropna默认丢弃任何含有缺失值的行。若要丢弃列，传入axis=1。

使用how参数：
    
    how='all'   丢弃全为NA的行
使用thresh参数：

    thresh=n    保留至少含有n个非空值的行

In [2]:
from numpy import nan as NA #将nan值以NA标记

data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [5]:
cleaned = data.dropna()
cleaned #默认丢弃任何含有NA的行（默认在axis=0上操作）。

,0,1,2
0,1.0,6.5,3.0


In [6]:
data.dropna(how='all') #丢弃全为NA的行。

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [7]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.623607,NaN,NaN
1,1.166220,NaN,NaN
2,-0.693397,NaN,0.256707
3,-0.555344,NaN,-0.827110
4,-1.246630,-0.435478,-0.776713
5,0.101593,-1.302912,-0.315744
6,0.380812,-1.322965,1.119984


In [8]:
df.dropna() #默认将会丢弃任何含有NA的行

,0,1,2
4,-1.246630,-0.435478,-0.776713
5,0.101593,-1.302912,-0.315744
6,0.380812,-1.322965,1.119984


In [9]:
df.dropna(thresh=2) #含有2个以上非空值的行将被保留。

,0,1,2
2,-0.693397,NaN,0.256707
3,-0.555344,NaN,-0.827110
4,-1.246630,-0.435478,-0.776713
5,0.101593,-1.302912,-0.315744
6,0.380812,-1.322965,1.119984


### 7.1.2 填充缺失数据

对于大多数情况而言，fillna方法是最主要的函数。

    通过一个【常数】调用fillna就会将缺失值替换为那个常数值。
    若是通过一个【字典】调用fillna，就可以实现对不同的列填充不同的值。
    fillna()的method参数还有：
        ffill(向前填充)/bfill(向后填充)，可选limit=n，即向前填充的行数

In [10]:
#对NA值使用常数填充
df.fillna(0) 

,0,1,2
0,-0.623607,0.000000,0.000000
1,1.166220,0.000000,0.000000
2,-0.693397,0.000000,0.256707
3,-0.555344,0.000000,-0.827110
4,-1.246630,-0.435478,-0.776713
5,0.101593,-1.302912,-0.315744
6,0.380812,-1.322965,1.119984


In [11]:
#通过字典dict调用fillna，可以实现不同的列填充不同的值(键对应列名，值对应填充值)。
df.fillna({1:99 , 2:100}) 

,0,1,2
0,-0.623607,99.000000,100.000000
1,1.166220,99.000000,100.000000
2,-0.693397,99.000000,0.256707
3,-0.555344,99.000000,-0.827110
4,-1.246630,-0.435478,-0.776713
5,0.101593,-1.302912,-0.315744
6,0.380812,-1.322965,1.119984


## 7.2 数据转换



### 7.2.1 移除重复数据

DataFrame的duplicated方法返回一个布尔型Series，表示各行是否是重复行（前面出现过的行）。
drop_duplicates方法丢弃（第二次及以后出现的）重复行。
    
       drop_duplicates方法的选择参数类似于DF选择列的方式，可以指定根据某列进行重复数据的处理。
       还有一个keep参数，可选keep='last'，从而保留最后一个重复值。

In [12]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [13]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

### 7.2.2 利用函数或映射进行数据转换

重要的函数：map()。可以理解为函数映射关系。

Series的map方法可以接受一个函数或含有映射关系的字典型对象

    注意map方式是Series的方法；可以接受函数，最常见的是lambda函数；可以接受字典对象

例如，添加一列食物的原材料（来源动物）

In [14]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [15]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}
#设定映射关系

注意到原数据中字母大小写不统一，使用Series的str.lower()方法将字母统一小写。

In [16]:
ndata=data['food'].str.lower() #将food列字母统一小写，以便跟映射中的键一一对应。
data['animal']=ndata.map(meat_to_animal) 
#新写入一列animal，使用data['animal']=；对Series对象使用map方法，可以理解为将Series按照映射关系生成新的Series。
#这里相当于将原Series即food映射成新Series即animal，映射关系由字典决定。
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [17]:
data['food'].map(lambda x: meat_to_animal[x.lower()])
#这里lambda函数：x形参对应到food列中的值，进行lower()方法的字母小写操作，然后map函数进行映射。

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.2.3 替换值

replace提供了一种实现替换的简单灵活的方式。注意，是Series的方法。

DF.replace(A,B):
    
    A,B可以各是一个值或一组值。A是被替换的值，B是替代值。
    可以是常数、列表和字典。

In [18]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [19]:
data.replace(-999,NA) #一对一的替换

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [20]:
data.replace([-999,-1000],0) #多对一的替换

0    1.0
1    0.0
2    2.0
3    0.0
4    0.0
5    3.0
dtype: float64

In [21]:
data.replace([-999,-1000],[NA,100]) #多对多的替换

0      1.0
1      NaN
2      2.0
3      NaN
4    100.0
5      3.0
dtype: float64

### 7.2.4 重命名轴索引

跟Series中的值一样（例如上面7.2.2的例子），轴标签也可以通过函数或映射进行转换（即轴索引也有map方法），从而得到一个新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构。

In [22]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [23]:
transform = lambda x: x[:4].upper() #将x指向的对象的前四行字母大写

data.index = data.index.map(transform) #将转换后的值赋予index
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


注意这里与7.2.2例子的异同点。出发点都是对数据进行添加或修改，方法内核都是函数转换。这种方法比逐个索引并进行修改要高效很多。

这里是对轴索引进行重命名操作，7.2.2对DF的列进行了数据转换。在表格结构上都属于“列”上的操作。

### 7.2.5 离散化和面元划分

为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。

重点使用pd.cut()和pd.qcut()函数，进行分组分析和分位数分析。它们均返回一个Categorical对象。

通俗的理解为，现在得到的元素不再是常见的一个数，而是【区间】。可以对区间计数、对区间命名。但在逻辑上要注意与原数据和分析目标的关联。

下面通过实例进行讲解。

（1）假设有一组人员数据，而你希望将它们划分为不同的年龄组：

In [24]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [25]:
bins = [18, 25, 35, 60, 100] #面元bin的创建
cats = pd.cut(ages, bins) #将各个数据划分到不同的区间，并且结果呈现的是区间本身。
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

pandas返回的是一个特殊的Categorical对象，结果展示了pandas.cut划分的面元。

可以将其看做一组表示面元名称的字符串。它的底层含有一个表示不同分类名称的类型数组，和一个codes属性中的年龄数据的标签（字符串的位置索引）：

In [26]:
cats.codes #年龄数据（区间这个字符串）的标签，即属于第几个面元。这里18-25的标签为0，25-35的标签为1,以此类推。

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [27]:
cats.categories #面元划分的细节信息。

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [28]:
pd.value_counts(cats) 
#pd.cut()的面元计数。注意思路的转换：表面上统计的是区间的个数，实际上对应到数据也就是数据落在这个区间里的个数。
#这种实现基于对每个数据都进行了区间对应。

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

（2）可以通过传递一个列表或数组到labels，设置自己的面元（区间）名称：

In [29]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names) #设置面元名称

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

（3）数据等分或分位数切割

①如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元。

In [30]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2) 
#传入面元的数量，而非准确边界。此时就会根据数据进行等分。
#precision表示小数点的精度。

[(0.77, 0.98], (0.13, 0.34], (0.55, 0.77], (0.77, 0.98], (0.77, 0.98], ..., (0.55, 0.77], (0.34, 0.55], (0.34, 0.55], (0.13, 0.34], (0.55, 0.77]]
Length: 20
Categories (4, interval[float64]): [(0.13, 0.34] < (0.34, 0.55] < (0.55, 0.77] < (0.77, 0.98]]

②qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。

根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元。

与cut类似，也可以传递自定义的分位数（0到1之间的数值，包含端点）：

In [31]:
data = np.random.randn(1000)  # 服从正态分布的数据
cats = pd.qcut(data, 4)  # 分位数切割，4等分。
cats

[(-0.714, 0.0115], (-2.82, -0.714], (0.704, 2.934], (-0.714, 0.0115], (0.0115, 0.704], ..., (0.0115, 0.704], (0.0115, 0.704], (-0.714, 0.0115], (0.704, 2.934], (0.704, 2.934]]
Length: 1000
Categories (4, interval[float64]): [(-2.82, -0.714] < (-0.714, 0.0115] < (0.0115, 0.704] < (0.704, 2.934]]

In [32]:
pd.value_counts(cats) #通过计数检查是否4等分。

(0.704, 2.934]      250
(0.0115, 0.704]     250
(-0.714, 0.0115]    250
(-2.82, -0.714]     250
dtype: int64

In [33]:
#自定义分位数
cats2 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

In [34]:
pd.value_counts(cats2)

(0.0115, 1.324]     400
(-1.345, 0.0115]    400
(1.324, 2.934]      100
(-2.82, -1.345]     100
dtype: int64

### 7.2.6 检测和过滤异常值

In [35]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.035204,-0.002331,0.056012,-0.027727
std,0.985872,1.005737,0.975349,0.977933
min,-3.111442,-3.634013,-3.032174,-3.458342
25%,-0.662202,-0.664737,-0.627530,-0.684643
50%,0.043105,-0.008718,0.064278,-0.037291
75%,0.730577,0.683381,0.764879,0.655272
max,2.896524,4.371172,2.866384,2.693177


In [36]:
data[(np.abs(data)>3).any(1)] #选出所有绝对值大于3的行。any()的参数为axis，即选轴。

,0,1,2,3
147,0.819512,-3.634013,1.002686,-0.362732
156,1.364016,-0.485552,0.214062,-3.345557
318,-3.111442,-1.374090,-0.674546,0.896826
467,0.819822,-0.541838,-2.122989,-3.095129
506,0.995381,-2.535642,2.088197,-3.458342
527,-0.990059,1.071454,-3.002567,-0.910521
659,0.043221,0.056332,-3.032174,0.150070
822,-1.679206,4.371172,-0.133228,0.427782
830,0.919191,3.197238,0.207003,-0.169964
958,-0.046565,3.188517,-0.103598,-1.318254


### 7.2.7 置换和随机抽样

numpy.random.permutation函数可实现对DataFrame中的Series或行进行置换（重新排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组。

要选出一个不含替代值（重复选择）的随机子集，可以使用sample方法。

In [37]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5) #假设我们要对行进行重排，DF有5行，即需要排列的轴长度为5。
sampler #生成一个可以用于表示新顺序的整数数组。

array([3, 0, 2, 4, 1])

In [38]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [39]:
df.take(sampler) #使用生成的新顺序数组sampler调用take函数，进行数据重排。这里默认按轴axis=0即行进行重排。

,0,1,2,3
3,12,13,14,15
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7


In [40]:
df.sample(n=3) #选择一个3行的子样本，不允许有重复选择。默认在axis=0即行上选择。

,0,1,2,3
4,16,17,18,19
2,8,9,10,11
0,0,1,2,3


In [41]:
df.sample(n=10,replace=True) #从DF中选择样本，允许有重复选择（inplace=True），因此样本量n可以大于原来的行数。

,0,1,2,3
1,4,5,6,7
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3
2,8,9,10,11
2,8,9,10,11
2,8,9,10,11
1,4,5,6,7
2,8,9,10,11


### 7.2.8 计算指标/虚拟变量

pandas有一个get_dummies函数用于实现该功能。

基本逻辑是，如果DF的一列有k个不同值，则可以衍生一个k列的值为1或0的矩阵或DF。相当于为每个唯一值生成一个虚拟变量，并返回一个DF。

In [42]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [43]:
dum = pd.get_dummies(df['key'],prefix='key') 
dum
#根据key列的每个唯一值都生成虚拟变量。返回的DF中，列名即是原key列的唯一值，并对原DF的每行进行判断。
#参数prefix为列添加前缀

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [44]:
df_with_dummy = df.join(dum) #在DF或Series的后面合并一个新pandas对象，使用pd.join(newpd)函数
df_with_dummy

,key,data1,key_a,key_b,key_c
0,b,0,0,1,0
1,b,1,0,1,0
2,a,2,1,0,0
3,c,3,0,0,1
4,a,4,1,0,0
5,b,5,0,1,0


将get_dummies与cut等离散化函数结合使用是统计应用的一个有用方法

In [45]:
np.random.seed(12345)

values = np.random.rand(10)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [46]:
cats=pd.cut(values,bins)
cats

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [47]:
pd.get_dummies(cats) 
#这里cats作为Categrical对象也可以传入。注意values虽然是数组，但其每个元素都有一个位置标签。0-9表示的是这10个数的位置。

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

## 7.3 字符串操作



### 7.3.1 字符串对象方法

内建的字符串方法十分丰富。详细见P209。

例如，一个逗号分隔的字符串可以使用split方法拆分成多块：

In [48]:
val = 'a,b,  guido'
val.split(',')
#split的参数为选择分隔符。

['a', 'b', '  guido']

split常和strip一起使用，用于清除空格（包括换行）：

In [49]:
pieces = [x.strip() for x in val.split(',')] #列表推导式。一个语句使用了两个函数。
pieces

['a', 'b', 'guido']

### 7.3.2 正则表达式

正则表达式提供了一种在文本中灵活查找或匹配（通常更为复杂）字符串模式的方式。正则表达式，常称作regex，是根据正则表达式语言编写的字符串。Python内置的re模块负责对字符串应用正则表达式。我将通过一些例子说明其使用方法。

re模块主要有三个主题：模式匹配、替代、拆分。关于正则表达式可以有单独的讲解，这里不详细讨论。

假设我们想将含有多种空白字符（制表符、空格、换行符）的字符串拆分开，描述一个或多个空白字符的正则表达式是 \s+

In [50]:
import re

In [51]:
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

调用re.split('\s+',text）时，正则表达式首先会被编译（compile），然后正则表达式的split方法在传入文本上被调用。可以使用re.compile自行编译，形成一个可复用的正则表达式对象。当要反复使用相同的正则表达式时，这一方法较为高效。

In [52]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

如果想获得所有匹配正则表达式的模式的列表，使用findall方法：

In [53]:
regex.findall(text) #寻找正则表达式使用的所有匹配模式

#注意，为了在正则表达式中避免转义符\的影响，可以使用原生字符串语法r'',或等价的'\\'。

['    ', '\t ', '  \t']

### 7.3.3 pandas中的向量化字符串函数

（暂略）

## 第8章

很多应用中，数据可能分布在多个文件或数据库中，抑或以某种不易于分析的格式进行排列。本章关注于对数据联合、连接以及重排列有用的工具。
## 8.1 分层索引

层次化索引允许在一个轴向上拥有多个（两个及以上）索引层级（相当于增加了维度）。笼统地说，层次化索引提供了一种在更低维度的形式中处理更高维度数据的方式。

理解层次化索引，要有[数据维度结构]的思维（层次化索引中哪一级别是高维度、哪一级别是低维度）。分别从Series和DataFrame介绍层次化索引。

### 8.1.1 Series的层次化索引

In [54]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data
# 多层索引结构

a  1    1.007189
   2   -1.296221
   3    0.274992
b  1    0.228913
   3    1.352917
c  1    0.886429
   2   -2.001637
d  2   -0.371843
   3    1.669025
dtype: float64

In [55]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单：

In [57]:
data[['b','c']] 
#可以尝试以下代码
#data['b']
#data['b':'c']
#data.loc[['b', 'd']]

b  1    0.228913
   3    1.352917
c  1    0.886429
   2   -2.001637
dtype: float64

In [58]:
data.loc[:,2] 
#对Series的逻辑来讲，这里是从“内层”开始索引。可以这样考虑，相对于DF来说，Series本来是一维对象，但在层次化索引中，出现了由外到内的索引层次。这里两层索引结构相当于增加了一个维度，因此loc方法的第一个A参数相当于选取所有的外层索引（在DF里是所有的行），第二个参数选取内层索引（DF里是所有的列）。

a   -1.296221
c   -2.001637
d   -0.371843
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。

例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中：

In [60]:
df=data.unstack()
df
#data本来是一维Series，但由于层次化索引增加了它的维度，事实上是一个二维数据结构。因此unstack()方法可以将层次化索引转换为单一索引结构。

,1,2,3
a,1.007189,-1.296221,0.274992
b,0.228913,NaN,1.352917
c,0.886429,-2.001637,NaN
d,NaN,-0.371843,1.669025


In [61]:
df.stack()
#unstack()的逆操作为stack()，即由单一索引结构转换为层次化索引结构。

a  1    1.007189
   2   -1.296221
   3    0.274992
b  1    0.228913
   3    1.352917
c  1    0.886429
   2   -2.001637
d  2   -0.371843
   3    1.669025
dtype: float64

### 8.1.2 DataFrame的层次化索引

DataFrame的每条轴都可以有层次化索引：

In [62]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

各层都可以有名字（可以是字符串，也可以是别的Python对象）。如果指定了名称，它们就会显示在控制台输出中：

In [65]:
frame.index.names = ['key1', 'key2'] #（行）索引名称
frame.columns.names = ['state', 'color'] #列名称
#在行列索引没有名称的时候，进行相应的调用可以使用默认位置，但要标明对应轴axis。

frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [66]:
frame['Ohio'] 
#不同于二维DF，这里层次化索引结构下的DF数据维度是更高维度的（4维）。因此索引的部分数据仍然是一个（3维）DF。

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

可以单独创建MultiIndex然后复用。上面那个DataFrame中的（带有分级名称）列可以这样创建：

MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

### 8.1.3 重排与分级排序

有时需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化）。经常结合sort_index(leve=)使用，可以实现索引层次的对调。

这里重要实现两点，第一是基于层次化索引的排序，第二是层次化索引的重新排列（并保持数据结构和数据内容不变）。

In [68]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [76]:
frame.swaplevel('key1', 'key2')
#通过swaplevel函数更换轴级别的顺序。也可以使用swaplevel(0,1)进行交换（基于默认位置）。注意返回的是新对象，原frame并未修改。

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [75]:
frame.sort_index(level=1) 

#level是multiindex的参数，是一个列表。可以通过df.index查看。这里根据第二个级别的索引进行排序。
#注意sort_index()是对单个索引排序。

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

交换级别时，常常也会用到sort_index，这样最终结果就是按照指定顺序进行字母排序了：

In [77]:
frame.swaplevel(0,1).sort_index(level=0)

#逐步分析：首先swaplevel(0,1)得到了out[76]的结果，即实现了初步的索引顺序转换；
#                其次sort_index(level=0)，根据新对象的0层次上的索引（即key2的数字1，2）进行排序。

#与原frame对比，实际上实现了索引顺序的对调，并保持了原有结构。
#这段代码是实现对调的基本代码。

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [78]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 8.1.4 按层级进行汇总统计

许多对DataFrame和Series的描述和汇总统计都有一个level选项，它用于指定在某条轴上求和的级别。以上面的DataFrame为例，我们可以根据行或列上的级别来进行求和。其实是利用了pandas的groupby功能。

In [79]:
frame.sum(level=1) 
#在key2级别上求和，即按照1,2来分别求和（1和2分别对各自的1a和1b求和）

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [80]:
frame.sum(level=0) 
#在key1级别上求和，即按照a,b来分别求和（a和b分别对各自的a1和a2求和）

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

In [81]:
frame.sum(level='color', axis=1)
#按照color进行求和，即color.Ohio和color.Colorado求和。注意这是列上的求和，要指定轴axis=1。
#color级别对应的默认位置是1，参数使用level=1也可。

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.5 使用DataFrame的列进行索引

通常我们不会使用DataFrame中一个或多个列作为行索引；反而可能想要将行索引移动到DataFrame的列中。

DF的set_index函数会生成一个新DataFrame对象，这个新对象使用[一个或多个列]作为索引：

reset_index是set_index的反操作，分层索引的索引层级会被移动到列中。

In [19]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [24]:
frame2 = frame.set_index(['c','d'])
frame2
#原DF为默认的整数索引。set_index函数选择一列或多列作为行索引。

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [25]:
frame2.reset_index()
#相当于把索引放入列中，并以默认的整数索引替代。

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 联合与合并数据集

pandas数据集的联合方法:

    pandas.merge根据一个或多个键将行进行连接。实现关系型数据库的[连接]操作。
    pandas.concat使对象在轴向上进行黏合或“堆叠”。
    combine_first允许将重叠的数据拼接在一起，以使用一个对象中的值填充另一个对象中的缺失值。

### 8.2.1 数据库风格的DataFrame连接

合并（merge）或连接（join）操作通过一个或多个键（key）连接行来联合数据集。这些运算是关系型数据库（基于SQL）的核心。pandas的merge函数是对数据应用这些算法的主要切入点。

merge的详细参数见P226。

In [27]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1
#多对一的合并

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [28]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [30]:
pd.merge(df1,df2,on='key')
#参数on用于指定连接键。如果列名不同，可以指定：left_on和right_on。

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [31]:
pd.merge(df1,df2,on='key',how='outer')

#参数how用以表明连接的方式。默认为inner（交集连接），left（左表键），right（右表键），outer（并集连接）。

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


在多对多的合并中，连接是行的笛卡儿积。即如果某元素在左表出现a次，右表出现b次，则合并结果中共出现a * b次。

In [32]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [33]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [34]:
pd.merge(df1, df2, on='key', how='left')
#按照左表进行连接。即按照左表中key列元素出现的顺序，进行笛卡儿积式的连接

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [35]:
pd.merge(df1, df2, on='key', how='inner')
#改变参数how，只是影响结果的显示，不改变笛卡儿积的合并逻辑。

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


### 8.2.2 根据索引合并

在某些情况下，DataFrame中用于合并的键是它的[索引]（上面讲的是列）。可以使用left_index=Ture和right_index=Ture来表示索引需要用来作为合并的键。

In [36]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
#两个DF需要合并的列一个在列中，一个是索引对象。

left1 #键在DF的列中

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [37]:
right1 #键在DF的索引中 

,group_val
a,3.5
b,7.0


In [38]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')
#根据左表key合并，右表根据索引合并

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


在多层索引数据的情况下，在索引上连接是一个隐式的多键合并：

In [39]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [40]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [43]:
df1 = pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')
#左表根据两层索引进行合并，因此传入一个列表

df1

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [46]:
df2 = df1.set_index(['key1','key2'])
df2

data  event1  event2
key1   key2                      
Ohio   2000   0.0     4.0     5.0
       2000   0.0     6.0     7.0
       2001   1.0     8.0     9.0
       2002   2.0    10.0    11.0
Nevada 2001   3.0     0.0     1.0
       2002   4.0     NaN     NaN
       2000   NaN     2.0     3.0

### 8.2.3 沿轴向连接

concat方法。默认在轴axis=0上生效。这里介绍DF上的使用


In [49]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
#将这两个DF在列方向上进行合并

In [50]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [51]:
df2

,three,four
a,5,6
c,7,8


In [53]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],sort=False)
#在列上进行合并，参数keys建立层次索引。

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [54]:
pd.concat([df1, df2],sort=False)
#如果在默认axis=0上合并，如下：

,one,two,three,four
a,0.0,1.0,NaN,NaN
b,2.0,3.0,NaN,NaN
c,4.0,5.0,NaN,NaN
a,NaN,NaN,5.0,6.0
c,NaN,NaN,7.0,8.0


In [59]:
pd.merge(df1,df2,left_index=True,right_index=True,how='outer')
#使用merge实现同样的效果，但是：（1）merge需要有相同的键进行合并，concat不需要（2）concat可以生成层次索引。

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


## 8.3 重塑和透视

重排表格型数据。

### 8.3.1 使用多层索引进行重塑

unstack 将列中的数据透视到行
stack 将行中的数据透视到列
需要对数据的维度、结构进行深度的理解。堆叠和拆堆的本质是基于行列索引或名称进行的数据结构重塑。

In [61]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [64]:
result = data.stack()
result
#将列透视到行，产生一个series（2维降1维）。注意列成为了层次索引的内层（列到行的转变）

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [63]:
data.stack().unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [69]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [66]:
data #比较可知，实际上完成了行列的互换。等价于data.T

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [67]:
data.T

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


在高维DF中，可以进行各种形式的重塑。实践性较强，了解方法后，可以实战积累。

In [70]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))

df #层次化索引的DF，三维。

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [71]:
df.unstack() #默认从内层（leve=1）进行拆堆，并维持了列的结构。

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [79]:
df.unstack(0)

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [80]:
df.unstack(0).stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### 8.3.2 将“长”透视为“宽”

多时间序列数据通常是以所谓的“长格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的。

我们先加载一些示例数据，做一些时间序列规整和数据清洗：

In [82]:
path = r'E:\Coding\Python\Python exercises\pydata-book-2nd-edition\examples\macrodata.csv'

In [89]:
data = pd.read_csv(path)
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [90]:
#目标：将上述数据转换成长数据，需要时间（精确到日）、realgdp/infl/ynemp三个数据（依年份排列）

#数据清洗和规整

periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
#datetime数据类型

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
#选择需要的几列数据，存为pd的index对象类型，并命名。

data = data.reindex(columns=columns)
#新data的创建

data.index = periods.to_timestamp('D', 'end')
#新data的索引为datetimeindex对象

pd.set_option('display.max_rows', 10)

In [97]:
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [100]:
data.stack()
#目的:将三列向一行转换（列转行的透视）

date                           item   
1959-03-31 23:59:59.999999999  realgdp     2710.349
                               infl           0.000
                               unemp          5.800
1959-06-30 23:59:59.999999999  realgdp     2778.801
                               infl           2.340
                                            ...    
2009-06-30 23:59:59.999999999  infl           3.370
                               unemp          9.200
2009-09-30 23:59:59.999999999  realgdp    12990.341
                               infl           3.560
                               unemp          9.600
Length: 609, dtype: float64

In [101]:
data.stack().reset_index()
# 8.1.3 使用DF的列进行索引 ： reset_index()将索引变成DF的列

,date,item,0
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [99]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
#给最后一列重命名

ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


转成常见的经济学数据格式（即每个不同item做一列）：

In [103]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

#前两个参数分别是作为行索引和列索引的列，第三个参数是填充DF的可选值。

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


### 8.3.2 将“宽”透视为“长”

pivot方法的反操作是pandas.melt，它将多列合并成一列。

In [104]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [105]:
pd.melt(df,['key'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9
